# 좌표 변환 함수

In [ ]:
import requests
import pandas as pd

# 1. 카카오 REST API 키 입력 (개인 키로 변경 필요)
KAKAO_API_KEY = "input your key"

# 2. 주소 → 좌표 변환 함수
def get_lat_lon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            lat = float(result["documents"][0]["y"])  # 위도
            lon = float(result["documents"][0]["x"])  # 경도
            return lat, lon
    return None, None  # 변환 실패 시

# 3. 주소 데이터프레임 생성
data = pd.read_csv('/content/kids_cafe_list_all.csv', encoding = 'cp949')

# 4. apply()를 활용하여 위도·경도 변환 적용
data["latitude"], data["longitude"] = zip(*data["주소"].apply(get_lat_lon))

# 5. 결과 출력
print(data.head)


<bound method NDFrame.head of                                키즈카페 이름  자치구  \
0                      서울형 키즈카페 시립 1호점  동작구   
1                   서울형 키즈카페 시립 뚝섬자벌레점  광진구   
2                      서울형 키즈카페 시립 목동점  양천구   
3                       서울형키즈카페 시립 옴팡점  노원구   
4                   서울형 키즈카페 강남구 역삼1동점  강남구   
..                                 ...  ...   
73                       은평아이맘놀이터 수색동점  은평구   
74  서울형 키즈카페 종로구 혜화동점(종로 혜명 아이들 상상놀이터)  종로구   
75              서울형 키즈카페 중구 중림동점(노리몽땅)   중구   
76      서울형 키즈카페 중랑구 망우본동점(중랑실내놀이터 양원)  중랑구   
77        서울형 키즈카페 중랑구 면목4동점(중랑 실내놀이터)  중랑구   

                                                   주소  이용연령_min  이용연령_max  \
0                서울특별시 동작구 노량진로 10 서울가족플라자 지하2층 (대방동)         4        10   
1                   서울특별시 광진구 강변북로 2202 2층 꿈틀나루 (자양동)         0         6   
2                  서울특별시 양천구 안양천로 1131 지식산업센터 2층 (목동)         3        12   
3                    서울특별시 노원구 동일로174길 27 서울생활사박물관 옴팡         0        13   
4            서울특별시 강

In [ ]:
data.to_csv('kids_cafe_list_all_xy.csv', index=False, encoding='cp949')

In [ ]:
import requests
import pandas as pd

# 1. 카카오 REST API 키 입력 (개인 키로 변경 필요)
KAKAO_API_KEY = "input your key"

# 2. 주소 → 좌표 변환 함수
def get_lat_lon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": address}

    response = requests.get(url, headers=headers, params=params)

    return response

a = get_lat_lon("서울특별시 송파구 백제고분로15길 9")
a


<Response [200]>

In [ ]:
b = a.json()
b

{'documents': [{'address': {'address_name': '서울 송파구 잠실동 230-1',
    'b_code': '1171010100',
    'h_code': '1171065000',
    'main_address_no': '230',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_h_name': '잠실본동',
    'region_3depth_name': '잠실동',
    'sub_address_no': '1',
    'x': '127.084298944796',
    'y': '37.5061594376588'},
   'address_name': '서울 송파구 백제고분로15길 9',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 송파구 백제고분로15길 9',
    'building_name': '잠실본동사무소',
    'main_building_no': '9',
    'region_1depth_name': '서울',
    'region_2depth_name': '송파구',
    'region_3depth_name': '잠실동',
    'road_name': '백제고분로15길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.084298944796',
    'y': '37.5061594376588',
    'zone_no': '05568'},
   'x': '127.084298944796',
   'y': '37.5061594376588'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

# 주변 장소 카운트

In [ ]:
import nest_asyncio
import asyncio
import aiohttp
import pandas as pd
from tqdm import tqdm

nest_asyncio.apply()  # 이미 실행 중인 이벤트 루프에 중첩 실행 허용

KAKAO_API_KEY = "input your key"

Category = ['대형마트', '편의점', '어린이집', '학교', '학원', '주차장', '주유소', '지하철역',
            '은행', '문화시설', '중개업소', '공공기관', '관광명소', '숙박', '음식점', '카페', '병원', '약국']
category_group_codes = ['MT1', 'CS2', 'PS3', 'SC4', 'AC5', 'PK6', 'OL7', 'SW8', 'BK9', 'CT1',
                        'AG2', 'PO3', 'AT4', 'AD5', 'FD6', 'CE7', 'HP8', 'PM9']
category_dict = dict(zip(Category, category_group_codes))
headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

# 🔁 카테고리별 요청
async def fetch_category(session, latitude, longitude, radius, category_name, category_code):
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    params = {
        "category_group_code": category_code,
        "x": longitude,
        "y": latitude,
        "radius": radius
    }

    try:
        async with session.get(url, headers=headers, params=params) as resp:
            data = await resp.json()
            documents = data.get("documents", [])
            count = data['meta'].get('total_count', 0)
            min_dist = min([int(doc.get("distance", "99999")) for doc in documents if doc.get("distance", "99999").isdigit()] or [99999])
            return {
                f"{category_name}_개수": count,
                f"{category_name}_최소거리(m)": min_dist
            }
    except Exception as e:
        print(f"⚠️ 카테고리 {category_name} 오류: {e}")
        return {
            f"{category_name}_개수": 0,
            f"{category_name}_최소거리(m)": 99999
        }

# 좌표 1개 처리
async def fetch_all_categories(session, latitude, longitude, radius):
    tasks = [
        fetch_category(session, latitude, longitude, radius, cat_name, cat_code)
        for cat_name, cat_code in category_dict.items()
    ]
    results = await asyncio.gather(*tasks)
    combined = {}
    for r in results:
        combined.update(r)
    return combined

# 전체 좌표 처리
async def analyze_all(df, radius):
    async with aiohttp.ClientSession() as session:
        results = []
        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"📍 반경 {radius}m 처리 중"):
            result = await fetch_all_categories(session, row.latitude, row.longitude, radius)
            results.append(result)
        return pd.concat([df, pd.DataFrame(results)], axis=1)

# 실행 함수
def run(radius):
    data = pd.read_csv(f'/content/drive/MyDrive/gwangjin_places_{radius}m.csv', encoding='cp949')
    loop = asyncio.get_event_loop()
    result_df = loop.run_until_complete(analyze_all(data, radius))
    result_df.to_csv(f'/content/drive/MyDrive/gwangjin_places_{radius}m_kakao.csv', encoding='cp949', index=False)
    print(result_df.head())

# 실행
for m in [500, 1000]:
    run(m)


📍 반경 500m 처리 중: 100%|██████████| 1586/1586 [04:43<00:00,  5.60it/s]


   Unnamed: 0       gid   longitude   latitude  \
0           0  다사608477  127.056911  37.528226   
1           1  다사608478  127.056906  37.529127   
2           2  다사609479  127.058032  37.530033   
3           3  다사609480  127.058027  37.530934   
4           4  다사609481  127.058022  37.531835   

                                            geometry  \
0  POLYGON ((127.05634807254776 37.52777310757020...   
1  POLYGON ((127.05634273423657 37.52867444283111...   
2  POLYGON ((127.05746915708907 37.52958002424764...   
3  POLYGON ((127.05746383192019 37.53048135936886...   
4  POLYGON ((127.05745850651408 37.53138269435151...   

                                        centroid   adm_cd  행자부행정동코드 시군구명  \
0  POINT (127.05691127388377 37.528225899148474)  1123078  11680565  강남구   
1    POINT (127.0569059422622 37.52912723440883)  1105067  11215847  광진구   
2   POINT (127.05803237873211 37.53003281047079)  1105067  11215847  광진구   
3  POINT (127.05802706025375 37.530934145591296)  1105067 

📍 반경 1000m 처리 중: 100%|██████████| 1586/1586 [05:12<00:00,  5.08it/s]

   Unnamed: 0       gid   longitude   latitude  \
0           0  다사608477  127.056911  37.528226   
1           1  다사608478  127.056906  37.529127   
2           2  다사609479  127.058032  37.530033   
3           3  다사609480  127.058027  37.530934   
4           4  다사609481  127.058022  37.531835   

                                            geometry  \
0  POLYGON ((127.05634807254776 37.52777310757020...   
1  POLYGON ((127.05634273423657 37.52867444283111...   
2  POLYGON ((127.05746915708907 37.52958002424764...   
3  POLYGON ((127.05746383192019 37.53048135936886...   
4  POLYGON ((127.05745850651408 37.53138269435151...   

                                        centroid   adm_cd  행자부행정동코드 시군구명  \
0  POINT (127.05691127388377 37.528225899148474)  1123078  11680565  강남구   
1    POINT (127.0569059422622 37.52912723440883)  1105067  11215847  광진구   
2   POINT (127.05803237873211 37.53003281047079)  1105067  11215847  광진구   
3  POINT (127.05802706025375 37.530934145591296)  1105067 

In [ ]:
import requests
import pandas as pd
import json

KAKAO_API_KEY = "input your key"

# 2. 카테고리 코드 매핑
Category = ['대형마트', '편의점', '어린이집', '학교', '학원', '주차장', '주유소', '지하철역',
            '은행', '문화시설', '중개업소', '공공기관', '관광명소', '숙박', '음식점', '카페', '병원', '약국']
category_group_codes = ['MT1', 'CS2', 'PS3', 'SC4', 'AC5', 'PK6', 'OL7', 'SW8', 'BK9', 'CT1',
                        'AG2', 'PO3', 'AT4', 'AD5', 'FD6', 'CE7', 'HP8', 'PM9']
category_dict = dict(zip(Category, category_group_codes))

def find_nearby_places(latitude, longitude, radius):
    """
    특정 좌표에서 반경 내 모든 카테고리의 장소 개수와 최소 거리 반환
    """
    url = "https://dapi.kakao.com/v2/local/search/category.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    result = {}

    # 원하는 그룹 코드 입력
    params = {
        "category_group_code": 'AC5',
        "x": longitude,
        "y": latitude,
        "radius": radius,
        "page": 1
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    print(data)

    return data



In [ ]:
test = find_nearby_places(37.51239909, 126.9273881, 500)

{'documents': [{'address_name': '서울 동작구 대방동 341-19', 'category_group_code': 'AC5', 'category_group_name': '학원', 'category_name': '교육,학문 > 학원', 'distance': '96', 'id': '10252981', 'phone': '02-826-7942', 'place_name': '대방전기통신학원', 'place_url': 'http://place.map.kakao.com/10252981', 'road_address_name': '서울 동작구 노량진로 4', 'x': '126.92630198640913', 'y': '37.51242598572201'}, {'address_name': '서울 동작구 대방동 339-1', 'category_group_code': 'AC5', 'category_group_name': '학원', 'category_name': '교육,학문 > 학원 > 공인중개사학원', 'distance': '141', 'id': '1381908228', 'phone': '02-815-0600', 'place_name': '에듀윌공인중개사 노량진학원', 'place_url': 'http://place.map.kakao.com/1381908228', 'road_address_name': '서울 동작구 노량진로 26', 'x': '126.928840786506', 'y': '37.5129339067207'}, {'address_name': '서울 동작구 대방동 339-1', 'category_group_code': 'AC5', 'category_group_name': '학원', 'category_name': '교육,학문 > 학원 > 주택관리사학원', 'distance': '150', 'id': '19138583', 'phone': '02-815-3388', 'place_name': '에듀윌 주택관리사 노량진학원', 'place_url': 'http:/

In [ ]:
test['meta'].get('total_count', [])

73

# 키워드 검색

## 예시

In [ ]:
import requests
import pandas as pd
import json

KAKAO_API_KEY = "input your key"

# 2. 카테고리 코드 매핑
Category = ['대형마트', '편의점', '어린이집', '학교', '학원', '주차장', '주유소', '지하철역',
            '은행', '문화시설', '중개업소', '공공기관', '관광명소', '숙박', '음식점', '카페', '병원', '약국']
category_group_codes = ['MT1', 'CS2', 'PS3', 'SC4', 'AC5', 'PK6', 'OL7', 'SW8', 'BK9', 'CT1',
                        'AG2', 'PO3', 'AT4', 'AD5', 'FD6', 'CE7', 'HP8', 'PM9']
category_dict = dict(zip(Category, category_group_codes))

def keyword_find_nearby(latitude, longitude, radius, query):

    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    result = {}

    # query에 원하는 키워드 입력
    params = {
        "x": longitude,
        "y": latitude,
        "radius": radius,
        "page": 1,
        "query": query
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    # print(data)

    return data



In [ ]:
# key = keyword_find_nearby(37.62017392, 127.0769362, 1000)
key = keyword_find_nearby(37.55678177, 127.1723165, 1000, '키즈카페')
key

{'documents': [{'address_name': '서울 강동구 상일동 526',
   'category_group_code': 'CE7',
   'category_group_name': '카페',
   'category_name': '가정,생활 > 유아 > 놀이시설 > 키즈카페 > 서울형키즈카페',
   'distance': '10',
   'id': '1796273347',
   'phone': '02-3425-7076',
   'place_name': '서울형키즈카페 상일2동점',
   'place_url': 'http://place.map.kakao.com/1796273347',
   'road_address_name': '서울 강동구 상일로12길 95',
   'x': '127.172338659013',
   'y': '37.5566902852837'},
  {'address_name': '서울 강동구 고덕동 693',
   'category_group_code': 'CE7',
   'category_group_name': '카페',
   'category_name': '가정,생활 > 유아 > 놀이시설 > 키즈카페 > 서울형키즈카페',
   'distance': '678',
   'id': '454095157',
   'phone': '02-3425-9435',
   'place_name': '서울형키즈카페 강동구 고덕2동점',
   'place_url': 'http://place.map.kakao.com/454095157',
   'road_address_name': '서울 강동구 고덕로 353',
   'x': '127.16472076448565',
   'y': '37.557683244852214'},
  {'address_name': '서울 강동구 상일동 산 77-27',
   'category_group_code': 'CE7',
   'category_group_name': '카페',
   'category_name': '가정,생활 >

In [ ]:
key['documents'][0]['category_name']

'여행 > 공원 > 도시근린공원'

## 카테고리 필터링

In [ ]:
def count_nearby_parks(latitude, longitude, radius):
    """
    주어진 좌표에서 반경 내 '도시근린공원'의 개수와 가장 가까운 공원의 이름과 거리(m)를 반환
    """
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    query = "공원"
    page = 1
    is_end = False

    count = 0
    min_distance = float('inf')
    closest_park_name = None

    while not is_end:
        params = {
            "x": longitude,
            "y": latitude,
            "radius": radius,
            "page": page,
            "query": query
        }

        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        documents = data.get("documents", [])

        for doc in documents:
            category = doc.get("category_name", "")
            distance_str = doc.get("distance", "").strip()
            place_name = doc.get("place_name", "")

            # 도시근린공원 카운트 및 최소 거리 체크
            if category == "여행 > 공원 > 도시근린공원":
                count += 1
                # print(place_name)
                try:
                    if distance_str.isdigit():
                        distance = int(distance_str)
                        if distance < min_distance:
                            min_distance = distance
                            closest_park_name = place_name
                except ValueError:
                    continue

        # 다음 페이지 여부 체크
        meta = data.get("meta", {})
        is_end = meta.get("is_end", True)
        page += 1

    # 결과 반환
    return {
        "공원_개수": count,
        "공원_최소거리(m)": min_distance if min_distance != float('inf') else None,
        "가까운_공원": closest_park_name

    }


In [ ]:
c = count_nearby_parks(37.63013826, 127.0414162, 1000)
c

{'공원_개수': 7, '공원_최소거리(m)': 148, '가까운_공원': '벌말어린이공원'}

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/gwangjin_places_1000m.csv', encoding='cp949')

def extract_park_info(row):
    result = count_nearby_parks(row.latitude, row.longitude, 1000)
    return result["공원_개수"], result["공원_최소거리(m)"], result["가까운_공원"]

data[['공원_개수', '공원_최소거리(m)', '가까운 공원']] = data.apply(extract_park_info, axis=1, result_type='expand')

data.to_csv(f'/content/drive/MyDrive/gwangjin_places_parks_1000m.csv', encoding='cp949', index=False)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/gwangjin_places_500m.csv', encoding='cp949')

def extract_park_info(row):
    result = count_nearby_parks(row.latitude, row.longitude, 500)
    return result["공원_개수"], result["공원_최소거리(m)"], result["가까운_공원"]

data[['공원_개수', '공원_최소거리(m)', '가까운 공원']] = data.apply(extract_park_info, axis=1, result_type='expand')

data.to_csv(f'/content/drive/MyDrive/gwangjin_places_parks_500m.csv', encoding='cp949', index=False)

## 검색어 개수

In [ ]:
import requests

def count_nearby_parks(latitude, longitude, radius):
    """
    주어진 좌표에서 반경 내 '키즈카페' 카테고리의 장소 중
    '서울형키즈카페'는 제외하고:
    - 유효 장소 개수
    - 가장 가까운 키즈카페 이름과 거리(m) (없으면 거리 99999)
    - 전체 검색 결과 개수, 제외 개수
    를 반환
    """
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    query = "키즈카페"
    page = 1
    is_end = False

    valid_count = 0
    exclude_count = 0
    min_distance = float('inf')
    closest_place_name = None

    while not is_end:
        params = {
            "x": longitude,
            "y": latitude,
            "radius": radius,
            "page": page,
            "query": query
        }

        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        documents = data.get("documents", [])

        for doc in documents:
            category = doc.get("category_name", "")
            distance_str = doc.get("distance", "").strip()
            place_name = doc.get("place_name", "").strip()

            # ✅ 제외 조건
            if any(exclude in category for exclude in ["음식점", "패션", "서비스", "서울형키즈카페"]):
                exclude_count += 1
                continue

            # ✅ 유효 장소 조건: category_name에 "키즈카페" 포함
            if "키즈카페" in category:
                valid_count += 1
                try:
                    if distance_str.isdigit():
                        distance = int(distance_str)
                        if distance < min_distance:
                            min_distance = distance
                            closest_place_name = place_name
                except ValueError:
                    continue

        meta = data.get("meta", {})
        is_end = meta.get("is_end", True)
        page += 1

    total_count = int(data.get("meta", {}).get("total_count", 0))

    return {
        f"{query}_전체개수": total_count - 1,
        "제외된_개수": exclude_count,
        "유효_개수": valid_count,
        f"{query}_최소거리(m)": min_distance if min_distance != float('inf') else 99999,
        f"가까운_{query}": closest_place_name if closest_place_name else "없음"
    }


In [7]:
# count_nearby_parks(37.63013826, 127.0414162, 1000)
count_nearby_parks(37.55678177, 127.1723165, 1000)


{'키즈카페_전체개수': 5,
 '제외된_개수': 3,
 '유효_개수': 3,
 '키즈카페_최소거리(m)': 53,
 '가까운_키즈카페': '꿀잼키즈룸 상일점'}

In [ ]:
for m in [500,1000]:
    data = pd.read_csv(f'/content/drive/MyDrive/gwangjin_places_{m}m.csv', encoding='cp949')

    query = '키즈카페'
    def extract_park_info(row):
        result = count_nearby_parks(row.latitude, row.longitude, 1000)
        return result[f"사설키즈_전체개수"], result['제외된_개수'], result['사설키즈_개수'], result[f"사설키즈_최소거리(m)"], result[f"가까운_{query}"]

    data[[f"사설키즈_전체개수", '제외된_개수', '사설키즈_개수', f"사설키즈_최소거리(m)", f"가까운_{query}"]] = data.apply(extract_park_info, axis=1, result_type='expand')

    data.to_csv(f'/content/drive/MyDrive/gwangjin_places_{query}_{m}.csv', encoding='cp949', index=False)

# 사설 키즈카페

In [ ]:
import requests

def count_nearby_parks(latitude, longitude, radius):
    """
    주어진 좌표에서 반경 내 '키즈카페' 카테고리의 장소 중
    '서울형키즈카페'는 제외하고:
    - 유효 장소 개수
    - 가장 가까운 키즈카페 이름과 거리(m) (없으면 거리 99999)
    - 전체 검색 결과 개수, 제외 개수
    를 반환
    """
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

    query = "키즈카페"
    page = 1
    is_end = False

    valid_count = 0
    exclude_count = 0
    min_distance = float('inf')
    closest_place_name = None

    while not is_end:
        params = {
            "x": longitude,
            "y": latitude,
            "radius": radius,
            "page": page,
            "query": query
        }

        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        documents = data.get("documents", [])

        for doc in documents:
            category = doc.get("category_name", "")
            distance_str = doc.get("distance", "").strip()
            place_name = doc.get("place_name", "").strip()

            # ✅ 제외 조건
            if any(exclude in category for exclude in ["음식점", "패션", "서비스", "서울형키즈카페"]):
                exclude_count += 1
                continue

            # ✅ 유효 장소 조건: category_name에 "키즈카페" 포함
            if "키즈카페" in category:
                valid_count += 1
                try:
                    if distance_str.isdigit():
                        distance = int(distance_str)
                        if distance < min_distance:
                            min_distance = distance
                            closest_place_name = place_name
                except ValueError:
                    continue

        meta = data.get("meta", {})
        is_end = meta.get("is_end", True)
        page += 1

    total_count = int(data.get("meta", {}).get("total_count", 0))

    return {
        f"사설키즈_전체개수": total_count - 1,
        "제외된_개수": exclude_count,
        "'사설키즈_개수'": valid_count,
        f"사설키즈_최소거리(m)": min_distance if min_distance != float('inf') else 99999,
        f"가까운_사설키즈": closest_place_name if closest_place_name else "없음"
    }


In [ ]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

KAKAO_API_KEY = "input your key"  # 여기에 실제 키 입력

def count_nearby_parks(latitude, longitude, radius=1000):
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    query = "키즈카페"
    page = 1
    is_end = False

    valid_count = 0
    exclude_count = 0
    min_distance = float('inf')
    closest_place_name = None

    while not is_end:
        params = {
            "x": longitude,
            "y": latitude,
            "radius": radius,
            "page": page,
            "query": query
        }

        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        documents = data.get("documents", [])

        for doc in documents:
            category = doc.get("category_name", "")
            distance_str = doc.get("distance", "").strip()
            place_name = doc.get("place_name", "").strip()

            if any(exclude in category for exclude in ["음식점", "패션", "서비스", "서울형키즈카페"]):
                exclude_count += 1
                continue

            if "키즈카페" in category:
                valid_count += 1
                if distance_str.isdigit():
                    distance = int(distance_str)
                    if distance < min_distance:
                        min_distance = distance
                        closest_place_name = place_name

        meta = data.get("meta", {})
        is_end = meta.get("is_end", True)
        page += 1

    total_count = int(data.get("meta", {}).get("total_count", 0))

    return [
        total_count - 1,
        exclude_count,
        valid_count,
        min_distance if min_distance != float('inf') else 99999,
        closest_place_name if closest_place_name else "없음"
    ]

# ✅ 멀티스레드 함수
def process_dataframe_in_parallel(df, radius=1000, max_workers=10):
    results = [None] * len(df)

    def task_wrapper(i, row):
        results[i] = count_nearby_parks(row.latitude, row.longitude, radius)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(task_wrapper, i, row) for i, row in df.iterrows()]
        for future in as_completed(futures):
            pass  # just wait

    return pd.DataFrame(results, columns=[
        "사설키즈_전체개수", "제외된_개수", "사설키즈_개수", "사설키즈_최소거리(m)", "가까운_사설키즈"
    ])

In [9]:
for m in [500, 1000]:
    df = pd.read_csv(f'/content/drive/MyDrive/gwangjin_places_{m}m.csv', encoding='cp949')
    new_features_df = process_dataframe_in_parallel(df, radius=1000, max_workers=20)
    result = pd.concat([df.reset_index(drop=True), new_features_df], axis=1)
    result.to_csv(f'/content/drive/MyDrive/gwangjin_places_키즈카페_{m}.csv', encoding='cp949', index=False)

In [10]:
for m in [500, 1000]:
    df = pd.read_csv(f'/content/drive/MyDrive/kids_cafe_list_all_xy.csv', encoding='cp949')
    new_features_df = process_dataframe_in_parallel(df, radius=1000, max_workers=20)
    result = pd.concat([df.reset_index(drop=True), new_features_df], axis=1)
    result.to_csv(f'/content/drive/MyDrive/kids_with_키즈카페_{m}m.csv', encoding='cp949', index=False)